In [1]:
!pip install boto3

### retrieval

In [4]:
# ============================================================
# 🔹 IMPORTS
# ============================================================
import boto3
import json
import os
import sys

# ============================================================
# 🔹 CONFIGURATION
# ============================================================
KNOWLEDGE_BASE_LIST = ["FXFXKMJZNW"]
QUERY_TEXT = "ปัจจัยที่จำเป็นในการดำรงชีวิตของสัตว์น้ำ"
NUMBER_OF_RESULTS = 5
AWS_REGION = "us-east-1"
OUTPUT_FILE = "kb_retrievals.json"

# ============================================================
# 🔹 FUNCTION: Retrieve results from multiple Knowledge Bases
# ============================================================
def retrieve_for_kbs(kb_list, query, num_results, region, profile_name=None):
    # Create session with profile if specified
    if profile_name:
        session = boto3.Session(profile_name=profile_name)
        client = session.client("bedrock-agent-runtime", region_name=region)
    else:
        client = boto3.client("bedrock-agent-runtime", region_name=region)

    out = {}

    for kb in kb_list:
        try:
            resp = client.retrieve(
                knowledgeBaseId=kb,
                retrievalQuery={"text": query},
                retrievalConfiguration={
                    "vectorSearchConfiguration": {"numberOfResults": num_results}
                },
            )

            hits = resp.get("retrievalResults", [])[:num_results]
            parsed = []

            for h in hits:
                loc = h.get("location", {}) or {}
                ltype = loc.get("type")

                if ltype == "S3":
                    source = loc.get("s3Location", {}).get("uri")
                elif ltype == "WEB":
                    source = loc.get("webLocation", {}).get("url")
                else:
                    source = loc

                parsed.append({
                    "text": h.get("content", {}).get("text", ""),
                    "source": source,
                    "score": h.get("score"),
                })

            out[kb] = {
                "query": query,
                "region": region,
                "count": len(parsed),
                "results": parsed,
            }

            print(f"✅ {kb}: {len(parsed)} results")

        except Exception as e:
            out[kb] = {"error": str(e)}
            print(f"❌ {kb}: error -> {e}")

    return out

# ============================================================
# 🔹 MAIN EXECUTION
# ============================================================
if __name__ == "__main__":
    results = retrieve_for_kbs(
        KNOWLEDGE_BASE_LIST,
        QUERY_TEXT,
        NUMBER_OF_RESULTS,
        AWS_REGION,
    )

    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"\n💾 Saved results to {OUTPUT_FILE}")

✅ FXFXKMJZNW: 5 results

💾 Saved results to kb_retrievals.json


### Comparison

In [5]:
import json

file_path = 'kb_retrievals.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [6]:
# ============================================================
# 🔹 IMPORTS
# ============================================================
import textwrap

# ============================================================
# 🔹 CONFIGURATION
# ============================================================
QUERY_TEXT = "ปัจจัยที่จำเป็นในการดำรงชีวิตของสัตว์น้ำ"

retrieved_data = data

#KNOWLEDGE_BASE_LIST = ["1UUZMNH8CP", "EBJHRCUOQ2", "MTZLVXWABU"]
knowledge_base_map = {
    "FXFXKMJZNW": "defaultchunk",
}

# ============================================================
# 🔹 DISPLAY RESULTS
# ============================================================
print("=========================================================")
print(f"🔎 QUERY TEXT: {QUERY_TEXT}")
print("=========================================================")

for kb_id, kb_name in knowledge_base_map.items():
    if kb_id in retrieved_data and retrieved_data[kb_id]["results"]:
        # ดึงข้อความจาก Chunk แรก (index 0)
        chunk_text = retrieved_data[kb_id]["results"][0]["text"]

        # จัดรูปแบบข้อความให้มีความกว้างสูงสุด 180 อักขระ
        wrapped_text = textwrap.fill(chunk_text, width=180)

        # แสดงผลลัพธ์
        print(f"\n📘 Knowledge Base ID: {kb_id} | Name: {kb_name}")
        print("--------------------------------------------------")
        print(wrapped_text)
    else:
        print(f"\n❌ ไม่พบข้อมูลสำหรับ ID: {kb_id} ({kb_name})")


🔎 QUERY TEXT: ปัจจัยที่จำเป็นในการดำรงชีวิตของสัตว์น้ำ

📘 Knowledge Base ID: FXFXKMJZNW | Name: defaultchunk
--------------------------------------------------
Barrow , 2002 ) ซึ่งลูกสัตว์น้ำส่วนใหญ่ต้องการอาหารเม็ดขนาด 0.1 มิลลิเมตร (Moren et al., 2011) ทั้งนี้ โดยทั่วไปการให้อาหารสัตว์น้ำระยะนี้จะเป็นการผสมผสานกันระหว่างอาหารมีชีวิต
กับอาหารที่ผลิตขึ้น (Conceiçao et al., 2010) อย่างไรก็ตามเนื่องจากข้อจำกัดบางประการของอาหารมีชีวิต เช่น ความ ยุ่งยากในการเพาะเลี้ยง มีราคาแพง รวมทั้งความแตกต่างของโภชนาการ (


## Example 2 คำถามเจาะจงข้อมูล (Specific/Factual Query)

In [ ]:
# ============================================================
# 🔹 IMPORTS
# ============================================================
import boto3
import json
import os
import sys

# ============================================================
# 🔹 CONFIGURATION
# ============================================================
KNOWLEDGE_BASE_LIST = ["FXFXKMJZNW"]
QUERY_TEXT = "อาหารลูกปลานิลควรมีโปรตีนกี่เปอร์เซ็นต์"
NUMBER_OF_RESULTS = 5
AWS_REGION = "us-east-1"
OUTPUT_FILE = "kb_retrievals2.json"

# ============================================================
# 🔹 FUNCTION: Retrieve results from multiple Knowledge Bases
# ============================================================
def retrieve_for_kbs(kb_list, query, num_results, region, profile_name=None):
    # Create session with profile if specified
    if profile_name:
        session = boto3.Session(profile_name=profile_name)
        client = session.client("bedrock-agent-runtime", region_name=region)
    else:
        client = boto3.client("bedrock-agent-runtime", region_name=region)

    out = {}

    for kb in kb_list:
        try:
            resp = client.retrieve(
                knowledgeBaseId=kb,
                retrievalQuery={"text": query},
                retrievalConfiguration={
                    "vectorSearchConfiguration": {"numberOfResults": num_results}
                },
            )

            hits = resp.get("retrievalResults", [])[:num_results]
            parsed = []

            for h in hits:
                loc = h.get("location", {}) or {}
                ltype = loc.get("type")

                if ltype == "S3":
                    source = loc.get("s3Location", {}).get("uri")
                elif ltype == "WEB":
                    source = loc.get("webLocation", {}).get("url")
                else:
                    source = loc

                parsed.append({
                    "text": h.get("content", {}).get("text", ""),
                    "source": source,
                    "score": h.get("score"),
                })

            out[kb] = {
                "query": query,
                "region": region,
                "count": len(parsed),
                "results": parsed,
            }

            print(f"✅ {kb}: {len(parsed)} results")

        except Exception as e:
            out[kb] = {"error": str(e)}
            print(f"❌ {kb}: error -> {e}")

    return out

# ============================================================
# 🔹 MAIN EXECUTION
# ============================================================
if __name__ == "__main__":
    results = retrieve_for_kbs(
        KNOWLEDGE_BASE_LIST,
        QUERY_TEXT,
        NUMBER_OF_RESULTS,
        AWS_REGION,
    )

    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"\n💾 Saved results to {OUTPUT_FILE}")

✅ FXFXKMJZNW: 5 results

💾 Saved results to kb_retrievals2.json


In [12]:
import json

file_path = 'kb_retrievals.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [13]:
# ============================================================
# 🔹 IMPORTS
# ============================================================
import textwrap

# ============================================================
# 🔹 CONFIGURATION
# ============================================================
QUERY_TEXT = "อาหารลูกปลานิลควรมีโปรตีนกี่เปอร์เซ็นต์"

retrieved_data = data

#KNOWLEDGE_BASE_LIST = ["1UUZMNH8CP", "EBJHRCUOQ2", "MTZLVXWABU"]
knowledge_base_map = {
    "FXFXKMJZNW": "defaultchunk",
}

# ============================================================
# 🔹 DISPLAY RESULTS
# ============================================================
print("=========================================================")
print(f"🔎 QUERY TEXT: {QUERY_TEXT}")
print("=========================================================")

for kb_id, kb_name in knowledge_base_map.items():
    if kb_id in retrieved_data and retrieved_data[kb_id]["results"]:
        # ดึงข้อความจาก Chunk แรก (index 0)
        chunk_text = retrieved_data[kb_id]["results"][0]["text"]

        # จัดรูปแบบข้อความให้มีความกว้างสูงสุด 180 อักขระ
        wrapped_text = textwrap.fill(chunk_text, width=180)

        # แสดงผลลัพธ์
        print(f"\n📘 Knowledge Base ID: {kb_id} | Name: {kb_name}")
        print("--------------------------------------------------")
        print(wrapped_text)
    else:
        print(f"\n❌ ไม่พบข้อมูลสำหรับ ID: {kb_id} ({kb_name})")


🔎 QUERY TEXT: อาหารลูกปลานิลควรมีโปรตีนกี่เปอร์เซ็นต์

📘 Knowledge Base ID: FXFXKMJZNW | Name: defaultchunk
--------------------------------------------------
Barrow , 2002 ) ซึ่งลูกสัตว์น้ำส่วนใหญ่ต้องการอาหารเม็ดขนาด 0.1 มิลลิเมตร (Moren et al., 2011) ทั้งนี้ โดยทั่วไปการให้อาหารสัตว์น้ำระยะนี้จะเป็นการผสมผสานกันระหว่างอาหารมีชีวิต
กับอาหารที่ผลิตขึ้น (Conceiçao et al., 2010) อย่างไรก็ตามเนื่องจากข้อจำกัดบางประการของอาหารมีชีวิต เช่น ความ ยุ่งยากในการเพาะเลี้ยง มีราคาแพง รวมทั้งความแตกต่างของโภชนาการ (


## Example3 คำถามเชิงแนวคิดและขั้นตอน (Conceptual/Method Query)


In [15]:
# ============================================================
# 🔹 IMPORTS
# ============================================================
import boto3
import json
import os
import sys

# ============================================================
# 🔹 CONFIGURATION
# ============================================================
KNOWLEDGE_BASE_LIST = ["FXFXKMJZNW"]
QUERY_TEXT = "อธิบายวิธีการคำนวณสูตรอาหารด้วยสี่เหลี่ยมของเพียร์สัน"
NUMBER_OF_RESULTS = 5
AWS_REGION = "us-east-1"
OUTPUT_FILE = "kb_retrievals3.json"

# ============================================================
# 🔹 FUNCTION: Retrieve results from multiple Knowledge Bases
# ============================================================
def retrieve_for_kbs(kb_list, query, num_results, region, profile_name=None):
    # Create session with profile if specified
    if profile_name:
        session = boto3.Session(profile_name=profile_name)
        client = session.client("bedrock-agent-runtime", region_name=region)
    else:
        client = boto3.client("bedrock-agent-runtime", region_name=region)

    out = {}

    for kb in kb_list:
        try:
            resp = client.retrieve(
                knowledgeBaseId=kb,
                retrievalQuery={"text": query},
                retrievalConfiguration={
                    "vectorSearchConfiguration": {"numberOfResults": num_results}
                },
            )

            hits = resp.get("retrievalResults", [])[:num_results]
            parsed = []

            for h in hits:
                loc = h.get("location", {}) or {}
                ltype = loc.get("type")

                if ltype == "S3":
                    source = loc.get("s3Location", {}).get("uri")
                elif ltype == "WEB":
                    source = loc.get("webLocation", {}).get("url")
                else:
                    source = loc

                parsed.append({
                    "text": h.get("content", {}).get("text", ""),
                    "source": source,
                    "score": h.get("score"),
                })

            out[kb] = {
                "query": query,
                "region": region,
                "count": len(parsed),
                "results": parsed,
            }

            print(f"✅ {kb}: {len(parsed)} results")

        except Exception as e:
            out[kb] = {"error": str(e)}
            print(f"❌ {kb}: error -> {e}")

    return out

# ============================================================
# 🔹 MAIN EXECUTION
# ============================================================
if __name__ == "__main__":
    results = retrieve_for_kbs(
        KNOWLEDGE_BASE_LIST,
        QUERY_TEXT,
        NUMBER_OF_RESULTS,
        AWS_REGION,
    )

    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"\n💾 Saved results to {OUTPUT_FILE}")

✅ FXFXKMJZNW: 5 results

💾 Saved results to kb_retrievals3.json


In [16]:
import json

file_path = 'kb_retrievals3.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [17]:
# ============================================================
# 🔹 IMPORTS
# ============================================================
import textwrap

# ============================================================
# 🔹 CONFIGURATION
# ============================================================
# QUERY_TEXT = "อาหารลูกปลานิลควรมีโปรตีนกี่เปอร์เซ็นต์"

retrieved_data = data

#KNOWLEDGE_BASE_LIST = ["1UUZMNH8CP", "EBJHRCUOQ2", "MTZLVXWABU"]
knowledge_base_map = {
    "FXFXKMJZNW": "defaultchunk",

}

# ============================================================
# 🔹 DISPLAY RESULTS
# ============================================================
print("=========================================================")
print(f"🔎 QUERY TEXT: {QUERY_TEXT}")
print("=========================================================")

for kb_id, kb_name in knowledge_base_map.items():
    if kb_id in retrieved_data and retrieved_data[kb_id]["results"]:
        # ดึงข้อความจาก Chunk แรก (index 0)
        chunk_text = retrieved_data[kb_id]["results"][0]["text"]

        # จัดรูปแบบข้อความให้มีความกว้างสูงสุด 180 อักขระ
        wrapped_text = textwrap.fill(chunk_text, width=180)

        # แสดงผลลัพธ์
        print(f"\n📘 Knowledge Base ID: {kb_id} | Name: {kb_name}")
        print("--------------------------------------------------")
        print(wrapped_text)
    else:
        print(f"\n❌ ไม่พบข้อมูลสำหรับ ID: {kb_id} ({kb_name})")


🔎 QUERY TEXT: อธิบายวิธีการคำนวณสูตรอาหารด้วยสี่เหลี่ยมของเพียร์สัน

📘 Knowledge Base ID: FXFXKMJZNW | Name: defaultchunk
--------------------------------------------------
วิธีสี่เหลี่ยมของเพียร์สัน และวิธีสมการทาง พีชคณิต จะเสียเวลามากและอาจผิดพลาดได้โดยเฉพาะอย่างยิ่งเมื่อใช้วัตถุดิบหลายชนิดในสูตรอาหาร
ตลอดจนราคาและคุณภาพของวัตถุดิบอาหารสัตว์น้ำมีเปลี่ยนแปลงอยู่เสมอขึ้นอยู่กับฤดูกาล และความ ต้องการ ดังนั้นเพื่อเป็นการแก้ไขปัญหาดังกล่าว
จึงได้มีการนำเอาเครื่องคอมพิวเตอร์มาช่วยในการ


## Example5 คำถามที่ต้องการข้อมูลจากตาราง (Table-based Query)

In [19]:
# ============================================================
# 🔹 IMPORTS
# ============================================================
import boto3
import json
import os
import sys

# ============================================================
# 🔹 CONFIGURATION
# ============================================================
KNOWLEDGE_BASE_LIST = ["FXFXKMJZNW"]
QUERY_TEXT = "ค่าเฉลี่ยของกากถั่วเหลืองที่แนะนำในสูตรอาหารปลากินเนื้อคือเท่าไหร่"
NUMBER_OF_RESULTS = 1
AWS_REGION = "us-east-1"
OUTPUT_FILE = "kb_retrievals5.json"

# ============================================================
# 🔹 FUNCTION: Retrieve results from multiple Knowledge Bases
# ============================================================
def retrieve_for_kbs(kb_list, query, num_results, region, profile_name=None):
    # Create session with profile if specified
    if profile_name:
        session = boto3.Session(profile_name=profile_name)
        client = session.client("bedrock-agent-runtime", region_name=region)
    else:
        client = boto3.client("bedrock-agent-runtime", region_name=region)

    out = {}

    for kb in kb_list:
        try:
            resp = client.retrieve(
                knowledgeBaseId=kb,
                retrievalQuery={"text": query},
                retrievalConfiguration={
                    "vectorSearchConfiguration": {"numberOfResults": num_results}
                },
            )

            hits = resp.get("retrievalResults", [])[:num_results]
            parsed = []

            for h in hits:
                loc = h.get("location", {}) or {}
                ltype = loc.get("type")

                if ltype == "S3":
                    source = loc.get("s3Location", {}).get("uri")
                elif ltype == "WEB":
                    source = loc.get("webLocation", {}).get("url")
                else:
                    source = loc

                parsed.append({
                    "text": h.get("content", {}).get("text", ""),
                    "source": source,
                    "score": h.get("score"),
                })

            out[kb] = {
                "query": query,
                "region": region,
                "count": len(parsed),
                "results": parsed,
            }

            print(f"✅ {kb}: {len(parsed)} results")

        except Exception as e:
            out[kb] = {"error": str(e)}
            print(f"❌ {kb}: error -> {e}")

    return out

# ============================================================
# 🔹 MAIN EXECUTION
# ============================================================
if __name__ == "__main__":
    results = retrieve_for_kbs(
        KNOWLEDGE_BASE_LIST,
        QUERY_TEXT,
        NUMBER_OF_RESULTS,
        AWS_REGION,
    )

    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"\n💾 Saved results to {OUTPUT_FILE}")

✅ FXFXKMJZNW: 1 results

💾 Saved results to kb_retrievals5.json


In [23]:
import json

file_path = 'kb_retrievals5.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
# ============================================================
# 🔹 RAG PIPELINE WITH CONCEPTUAL QUERY
# ============================================================
import boto3
import json

def rag_pipeline(query, knowledge_base_id, num_results=5):
    """
    Complete RAG pipeline: Retrieve + Generate
    
    Args:
        query: User's question
        knowledge_base_id: AWS Bedrock Knowledge Base ID
        num_results: Number of chunks to retrieve
    
    Returns:
        dict: Contains query, retrieved chunks, and generated answer
    """
    # Initialize clients
    bedrock_agent = boto3.client("bedrock-agent-runtime", region_name="us-east-1")
    bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")
    
    print("=" * 80)
    print(f"🔍 Query: {query}")
    print("=" * 80)
    
    # Step 1: Retrieve from Knowledge Base
    retrieve_resp = bedrock_agent.retrieve(
        knowledgeBaseId=knowledge_base_id,
        retrievalQuery={"text": query},
        retrievalConfiguration={
            "vectorSearchConfiguration": {"numberOfResults": num_results}
        },
    )
    
    results = retrieve_resp.get("retrievalResults", [])
    contexts = [r.get("content", {}).get("text", "") for r in results]
    
    print(f"\n✅ Retrieved {len(results)} chunks")
    
    # Step 2: Build prompt with context
    context_text = "\n\n".join([f"[Document {i+1}]\n{txt}" for i, txt in enumerate(contexts)])
    
    prompt = f"""You are an expert assistant in aquaculture and aquatic animal husbandry. Answer questions based ONLY on the provided context from Thai aquaculture documentation.

Reference Information:
{context_text}

Question: {query}

Instructions:
- Answer in Thai language only
- Base your answer strictly on the provided context
- Be clear, concise, and accurate
- If the context doesn't contain enough information to answer, state this clearly
- Use proper aquaculture terminology in Thai

Answer:"""
    
    # Step 3: Generate answer with Claude
    request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 2000,
        "temperature": 0.5,
        "messages": [{"role": "user", "content": prompt}]
    }
    
    response = bedrock.invoke_model(
        modelId="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
        body=json.dumps(request)
    )
    
    answer = json.loads(response['body'].read())['content'][0]['text']
    
    print("\n" + "=" * 80)
    print("🤖 ANSWER FROM CLAUDE SONNET 4.5:")
    print("=" * 80)
    print(answer)
    print("=" * 80 + "\n")
    
    return {
        "query": query,
        "retrieved_chunks": len(results),
        "answer": answer
    }

# Test with different query
result = rag_pipeline(
    query="อธิบายวิธีการคำนวณสูตรอาหารด้วยสี่เหลี่ยมของเพียร์สัน",
    knowledge_base_id="FXFXKMJZNW"
)